In [9]:
import pandas as pd
from pymongo import MongoClient
import time

client = MongoClient('localhost', 27017)
db = client['ztbd_with_aggregation']
collection = db['lego_collection']

# SELECTY

In [10]:
#Select z parzystymi num_parts dla setu (zamiast #funkcja do pobierania kolorów o parzystych ID - 10 rekordów)
start_time = time.time()

result = db.lego_collection.find({"num_parts": {"$mod": [2, 0]}}, {"set_number": 1, "num_parts": 1, "_id": 0}).limit(10)

end_time = time.time()
for document in result:
    print(document)
print(end_time - start_time)

{'set_number': '00-4', 'num_parts': 126}
{'set_number': '0011-2', 'num_parts': 12}
{'set_number': '0012-1', 'num_parts': 12}
{'set_number': '0013-1', 'num_parts': 12}
{'set_number': '0014-1', 'num_parts': 12}
{'set_number': '0015-1', 'num_parts': 68}
{'set_number': '022-1', 'num_parts': 110}
{'set_number': '03093-1', 'num_parts': 70}
{'set_number': '055-2', 'num_parts': 256}
{'set_number': '080-1', 'num_parts': 710}
0.0009949207305908203


In [13]:
#funkcja do pobierania wszystkich kolorów - 100 rekordów
start_time = time.time()

result = db.lego_collection.aggregate([
        {"$unwind": "$parts"},
        {"$group": {"_id": "$parts.color"}},
        {"$limit": 100}
    ])

end_time = time.time()
for document in result:
    print(document)
print(end_time - start_time)

{'_id': 'Trans-Very Lt Blue'}
{'_id': 'Trans-Bright Green'}
{'_id': 'Medium Lavender'}
{'_id': 'Light Lime'}
{'_id': 'Trans-Dark Pink'}
{'_id': 'Salmon'}
{'_id': 'Light Turquoise'}
{'_id': 'Glitter Trans-Clear'}
{'_id': 'Blue-Violet'}
{'_id': 'Flesh'}
{'_id': 'Glitter Trans-Light Blue'}
{'_id': 'Bright Pink'}
{'_id': 'Dark Purple'}
{'_id': 'Light Violet'}
{'_id': 'Medium Orange'}
{'_id': 'Dark Azure'}
{'_id': 'White'}
{'_id': 'Medium Violet'}
{'_id': 'Speckle Black-Gold'}
{'_id': 'Lavender'}
{'_id': 'Trans-Black'}
{'_id': 'Copper'}
{'_id': 'Sand Green'}
{'_id': 'Glow In Dark Trans'}
{'_id': 'Trans-Orange'}
{'_id': 'Sand Blue'}
{'_id': 'Pearl Dark Gray'}
{'_id': 'Orange'}
{'_id': 'Dark Red'}
{'_id': 'Reddish Brown'}
{'_id': '[No Color]'}
{'_id': 'Pearl Light Gold'}
{'_id': 'Pearl Gold'}
{'_id': 'Light Blue'}
{'_id': 'Trans-Green'}
{'_id': 'Black'}
{'_id': 'Medium Dark Pink'}
{'_id': 'Metal Blue'}
{'_id': 'Light Salmon'}
{'_id': 'Chrome Antique Brass'}
{'_id': 'Glow In Dark Opaque'}
{'_i

In [14]:
# odczytanie setow z poszczególnych lat + sortowanie malejąco według liczby części 
start_time = time.time()

pipeline = [
        {"$group": {"_id": "$year", "sets": {"$push": {"set_number": "$set_number", "num_parts": "$num_parts"}}}},
        {"$sort": {"_id": -1}},
        {"$unwind": "$sets"},
        {"$sort": {"sets.num_parts": -1}},
        {"$group": {"_id": "$_id", "sets": {"$push": "$sets"}}},
        {"$project": {"year": "$_id", "sets": 1, "_id": 0}},
        {"$sort": {"year": -1}}
    ]
result = db.lego_collection.aggregate(pipeline)

end_time = time.time()
for document in result:
    print(document)
print(end_time - start_time)

{'sets': [{'set_number': '10255-1', 'num_parts': 4009}, {'set_number': '10255-1', 'num_parts': 4009}, {'set_number': '10257-1', 'num_parts': 2669}, {'set_number': '10257-1', 'num_parts': 2669}, {'set_number': '71042-1', 'num_parts': 2286}, {'set_number': '71042-1', 'num_parts': 2286}, {'set_number': '21309-1', 'num_parts': 1969}, {'set_number': '21309-1', 'num_parts': 1969}, {'set_number': '75144-1', 'num_parts': 1702}, {'set_number': '75144-1', 'num_parts': 1702}, {'set_number': '70912-1', 'num_parts': 1621}, {'set_number': '70912-1', 'num_parts': 1621}, {'set_number': '40174-1', 'num_parts': 1450}, {'set_number': '40174-1', 'num_parts': 1450}, {'set_number': '42064-1', 'num_parts': 1326}, {'set_number': '42064-1', 'num_parts': 1326}, {'set_number': '42066-1', 'num_parts': 1150}, {'set_number': '42066-1', 'num_parts': 1150}, {'set_number': '70627-1', 'num_parts': 1118}, {'set_number': '70627-1', 'num_parts': 1118}, {'set_number': '70916-1', 'num_parts': 1052}, {'set_number': '70916-1'

In [15]:
# Odczytanie części z poszczególnych/konkretnej kategorii (setu)
start_time = time.time()

result = db.lego_collection.find_one({"set_number": "0015-1"})

end_time = time.time()
# for document in result:
#     print(document)
print(result)
print(end_time - start_time)

{'_id': ObjectId('663791f229182c7f1b44cba2'), 'set_number': '0015-1', 'version': 1, 'year': 1979, 'theme_name': 'Updated Classic', 'num_parts': 68, 'parts': [{'part_num': '3626apr0001', 'name': 'Minifig Head Standard Grin Print [Solid Stud]', 'category_name': 'Minifigs', 'color': 'green', 'rgb': 'F2CD37', 'is_trans': False, 'quantity': 8, 'is_spare': False}, {'part_num': '3838', 'name': 'Minifig Airtanks', 'category_name': 'Minifig Accessories', 'color': 'green', 'rgb': 'FFFFFF', 'is_trans': False, 'quantity': 2, 'is_spare': False}, {'part_num': '3838', 'name': 'Minifig Airtanks', 'category_name': 'Minifig Accessories', 'color': 'green', 'rgb': 'F2CD37', 'is_trans': False, 'quantity': 2, 'is_spare': False}, {'part_num': '3838', 'name': 'Minifig Airtanks', 'category_name': 'Minifig Accessories', 'color': 'green', 'rgb': 'C91A09', 'is_trans': False, 'quantity': 2, 'is_spare': False}, {'part_num': '3842a', 'name': 'Minifig Helmet Classic with Thin Chin Guard and Visor Dimples', 'category_

In [16]:
# najpopularniejsze motywy + ilosc zestawow w tym temacie 
start_time = time.time()

pipeline = [
    {"$group": {"_id": "$theme_name", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}  # Możesz zmienić wartość limitu, aby pobrać więcej lub mniej wyników
]

result = db.lego_collection.aggregate(pipeline)
end_time = time.time()
for document in result:
    print(document)
#print(result)
print(end_time - start_time)

{'_id': 'updated_theme', 'count': 5549}
{'_id': 'Construction', 'count': 366}
{'_id': 'Airport', 'count': 338}
{'_id': 'Police', 'count': 314}
{'_id': 'Traffic', 'count': 302}
{'_id': 'Star Wars Episode 4/5/6', 'count': 268}
{'_id': 'Bulk Bricks', 'count': 250}
{'_id': 'Fire', 'count': 232}
{'_id': 'Race', 'count': 218}
{'_id': 'Basic', 'count': 210}
0.05500459671020508


# UPDATE

In [ ]:
# Złożona operacja aktualizacji:
#Dla jednego setu:
# Zmienić kolor wszystkich części na "green".
# Zaktualizować num_parts na num_parts + 50.
# Zaktualizować quantity pierwszego elementu na quantity * 2.
# Zmienić theme_name na "Updated Classic".
start_time = time.time()

# Pobieramy dokument, który będziemy aktualizować
document = db.lego_collection.find_one({"set_number": "0015-1"})

if document:
    updated_parts = []
    for part in document["parts"]:
        updated_part = part.copy()
        updated_part["color"] = "green"  # Zmiana koloru wszystkich części
        updated_parts.append(updated_part)
    
    if updated_parts:
        updated_parts[0]["quantity"] *= 2  # Zmiana ilości pierwszego elementu

    # Aktualizacja dokumentu za pomocą findOneAndUpdate
    result = db.lego_collection.find_one_and_update(
        {"set_number": "0015-1"},
        {
            "$set": {
                "num_parts": document["num_parts"] + 50,  # Aktualizacja liczby części
                "parts": updated_parts,  # Aktualizacja listy części
                "theme_name": "Updated Classic"  # Aktualizacja nazwy motywu
            }
        },
        #return_document=pymongo.ReturnDocument.AFTER  # Opcjonalnie, aby zwrócić zaktualizowany dokument
    )

end_time = time.time()
print(f"Updated document: {result}")
print(f"Time taken: {end_time - start_time} seconds")

In [ ]:
#Update # aktualizacja zestawów z poszczególnych lat

start_time = time.time()

# Znalezienie zestawów z roku 2024
sets_to_update = db.lego_collection.find({"year": 2024})

# Iteracja przez znalezione zestawy i aktualizacja ich roku na 2025
for lego_set in sets_to_update:
    lego_set['year'] = 2025
    db.lego_collection.replace_one({"_id": lego_set['_id']}, lego_set)
end_time = time.time()
#print(result.modified_count)
print(end_time - start_time)

In [ ]:
#Update najpopularniejszych motywów (zmiana nazwy motywu na updated_theme):
start_time = time.time()

pipeline = [
    {"$group": {"_id": "$theme_name", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

result = db.lego_collection.aggregate(pipeline)

for document in result:
    theme_name = document["_id"]
    db.lego_collection.update_many(
        {"theme_name": theme_name},
        {"$set": {"theme_name": "updated_theme"}}
    )

end_time = time.time()
print(end_time - start_time)

# DELETE

In [ ]:
def delete_records(num_records):
    records = db.lego_collection.find().limit(num_records)
    deleted_count = 0
    for record in records:
        db.lego_collection.delete_one({"_id": record["_id"]})
        deleted_count += 1
    return deleted_count
for i in [10, 100, 1000]:
    start_time = time.time()
    delete_records(i)
    end_time = time.time()
    print(end_time - start_time)

In [ ]:
#Delete calosc
start_time = time.time()
result = db.lego_collection.delete_many({})
end_time = time.time()
print(f"Usunięto {result.deleted_count} dokumentów.")
print(end_time - start_time)
